In [3]:
import cv2
import numpy as np

def get_edge_img(color_img, gaussian_ksize=5, gaussian_sigmax=1,
                 canny_threshold1=50, canny_threshold2=100):
    """
    对输入的彩色图像进行高斯滤波和Canny边缘检测。
    
    :param color_img: 输入的彩色图像
    :param gaussian_ksize: 高斯核大小
    :param gaussian_sigmax: 高斯核标准差
    :param canny_threshold1: Canny边缘检测的第一个阈值
    :param canny_threshold2: Canny边缘检测的第二个阈值
    :return: 边缘检测后的图像
    """
    # 高斯模糊
    gaussian = cv2.GaussianBlur(color_img, (gaussian_ksize, gaussian_ksize), gaussian_sigmax)
    # 灰度转换
    gray_img = cv2.cvtColor(gaussian, cv2.COLOR_BGR2GRAY)
    # Canny 边缘检测
    edge_img = cv2.Canny(gray_img, canny_threshold1, canny_threshold2)
    return edge_img

def roi_mask(gray_img):
    """
    应用感兴趣区域掩码。
    
    :param gray_img: 灰度图像
    :return: 应用掩码后的图像
    """
    poly_pts = np.array([[[0, 368], [300, 210], [340, 210], [640, 368]]])
    mask = np.zeros_like(gray_img)
    cv2.fillPoly(mask, pts=poly_pts, color=255)
    img_mask = cv2.bitwise_and(gray_img, mask)
    return img_mask

def get_lines(edge_img):

    """
    从边缘图像中提取直线，并进行筛选和拟合。
    
    :param edge_img: 边缘图像
    :return: 左右车道线的两点坐标
 
    1. 函数的输入为边缘图像(edge_img),目的是从中提取左右车道线的两点坐标。

    2. 使用HoughLines函数对边缘图像进行霍夫变换,检测出一组直线。函数参数包括距离分辨率、角度分辨率、投票阈值、最小线段长度和最大线段间隙。

    3. 根据直线的斜率将其分为左车道线和右车道线两组。斜率小于0的为左车道线,斜率大于0的为右车道线。

    4. 对左右车道线分别进行异常值剔除。计算每组直线的平均斜率,并找出与平均斜率差异最大的直线。如果差异超过给定阈值(这里为0.2),则将该直线剔除,重复此过程直到没有异常值或直线数量为0。

    5. 对剔除异常值后的左右车道线分别进行最小二乘拟合,得到拟合直线的两个端点坐标。使用numpy的polyfit()函数进行一阶多项式拟合,并计算拟合直线在x坐标最小值和最大值处的y坐标,得到两个端点。如果左车道线或右车道线为空,则对应的拟合结果为None。

    通过这个函数,可以从边缘图像中提取出左右车道线,并对其进行异常值剔除和最小二乘拟合,得到表示车道线的两点坐标。这对于实现车道线检测和跟踪非常有用,是自动驾驶中的重要环节之一。
 
 
    """

    return left_fit, right_fit


def draw_line(img, lines):
    """
    在图像上绘制车道线。
    
    :param img: 输入的彩色图像
    :param lines: 左右车道线的坐标
    """
    left_line, right_line = lines
    if left_line is not None:
        cv2.line(img, tuple(left_line[0]), tuple(left_line[1]), color=(0, 255, 255), thickness=5)
    if right_line is not None:
        cv2.line(img, tuple(right_line[0]), tuple(right_line[1]), color=(0, 255, 255), thickness=5)

def show_lane(color_img):
    """
    显示车道线检测结果。
    
    :param color_img: 输入的彩色图像
    :return: 绘制车道线后的图像
    """
    edge_img = get_edge_img(color_img)
    mask_gray_img = roi_mask(edge_img)
    lines = get_lines(mask_gray_img)
    draw_line(color_img, lines)
    return color_img

if __name__ == '__main__':
    capture = cv2.VideoCapture('drive.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    outfile = cv2.VideoWriter('output.avi', fourcc, 25, (1280, 368))

    while True:
        ret, frame = capture.read()
        if not ret:
            break
        origin = np.copy(frame)
        frame = show_lane(frame)
        output = np.concatenate((origin, frame), axis=1)
        outfile.write(output)
        cv2.imshow('video', output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    capture.release()
    outfile.release()
    cv2.destroyAllWindows()